Not a tutorial! This notebook just generates the fake data used in some of the tutorials.

In [9]:
import pickle

from astropy.time import Time
import astropy.units as u
from astropy.table import QTable
import numpy as np
%matplotlib inline

from twobody import KeplerOrbit, PolynomialRVTrend
from thejoker import RVData

In [4]:
n_data = 256

rnd = np.random.RandomState(42)

t0 = Time('2019-1-1') + rnd.uniform(0., 40) * u.day

truth1 = dict()
truth1['P'] = np.random.uniform(40, 80) * u.day
truth1['M0'] = rnd.uniform(0., 2*np.pi) * u.radian
truth1['omega'] = rnd.uniform(0., 2*np.pi) * u.radian
truth1['e'] = 0.1
truth1['K'] = rnd.uniform(5, 15) * u.km/u.s
truth1['v0'] = rnd.uniform(-50, 50) * u.km/u.s

orbit = KeplerOrbit(P=truth1['P'], e=truth1['e'], omega=truth1['omega'],
                    M0=truth1['M0'], t0=t0, K=truth1['K'],
                    i=90*u.deg, Omega=0*u.deg,  # these don't matter
                    barycenter=PolynomialRVTrend([truth1['v0']]))

t = t0 + truth1['P'] * np.sort(rnd.uniform(0, 3., n_data))

rv = orbit.radial_velocity(t)
err = 10**np.random.uniform(-1, 0.5, size=len(rv)) * u.km/u.s
data = RVData(t, rv, rv_err=err)

In [12]:
tbl = QTable()
tbl['bjd'] = data.t.tcb.jd
tbl['rv'] = data.rv
tbl['rv_err'] = data.rv_err
tbl.write('data.ecsv', overwrite=True)

In [14]:
with open('true-orbit.pkl', 'wb') as f:
    pickle.dump(orbit.elements, f)

In [ ]:
orbit.elements.